# pyBela tutorial 2: Monitor
This tutorial expects the `potentiometers` project to be running on Bela. If the Bela is connected to your laptop, you can run the cell below to copy the `potentiometers` code onto your Bela and run it:

In [ ]:
!scp -r ../bela-code/potentiometers root@bela.local:Bela/projects
!ssh root@bela.local "make -C Bela stop Bela PROJECT=potentiometers run"

You will also need to connect two potentiometers to Bela analog inputs 0 and 1. Instructions on how to do so and some details on the Bela code are given in the notebook `1_Streamer.ipynb`.

This notebook is a tutorial for the Monitor class in the pyBela python library. The monitor allows you to "take a look" at variables in your Bela code. By taking a look we mean either requesting a single value (*what value does `pot1` have right now?*) or sampling the value of a variable, that is, getting a value every number of frames (*can you tell me the value of `pot1` every 1000 frames?*). The monitor can be useful to calibrate sensors or, in general, debug your Bela code. 

The Monitor class inherits from the Streamer, so you can use it in a similar way, with the only difference that you will need to specify a monitoring period. Let's take a look at an example:

In [2]:
import asyncio
import os
import pandas as pd
from pyBela import Monitor

# this environment variable allows displaying the bokeh plots. If you are in vscode, you need to set this to the vscode port:
os.environ["BOKEH_ALLOW_WS_ORIGIN"] ="1t4j54lsdj67h02ol8hionopt4k7b7ngd9483l5q5pagr3j2droq" 
# alternatively, you can set it to the jupyter notebook port:
# os.environ["BOKEH_ALLOW_WS_ORIGIN"]="127.0.0.1:8888" # if in jupyter notebook -- replace with the port number of your notebook

As with the streamer, we need to instantiate the Monitor class and run the `connect()` method to establish the websocket connection with Bela. If the connection fails, make sure Bela is plugged in to your laptop and that the `potentiometer` project is running on Bela. 

In [3]:
monitor = Monitor()
monitor.connect()

'Connection successful'

### Using asyncio to monitor data for a fixed amount of time

The monitor works in a very similar way to the Streamer but with a key difference: it doesn't continuously stream the values of the variables. Instead, it samples and streams these values periodically based on the given period. In this example, we will be monitoring two variables, `pot1` and `pot2` with a period 1000. 

As with the Streamer example, we can use asyncio to time the monitoring session:

In [4]:
monitor.start_monitoring(variables= ["pot1", "pot2"], 
                         periods= [1000,2000])
await asyncio.sleep(2)
monitored_values = monitor.stop_monitoring()

Started monitoring variables ['pot1', 'pot2']... Run stop_monitoring() to stop monitoring.
Stopped monitoring variables ['pot1', 'pot2']...


### Retrieving the data

`stop_monitoring` returns a dictionary with the monitored values and its timestamps. You can also access this data in `monitor.values`. Note that the timestamps are spaced by the period as we requested.

In [5]:
df = pd.DataFrame(monitored_values["pot1"])
df.head()


,timestamps,values
0,837152,0.451065
1,838152,0.451050
2,839152,0.451080
3,840152,0.451019
4,841152,0.451096


In [6]:
df = pd.DataFrame(monitored_values["pot2"])
df.head()

,timestamps,values
0,837152,0.493271
1,839152,0.493286
2,841152,0.493179
3,843152,0.493225
4,845152,0.493256


A note regarding the periods: in this example, the returned timestamps correspond to the analog frames elapsed in the Bela code. Let's take a look at the `render` loop in the `potentiometers` Bela project:

```cpp
void render(BelaContext *context, void *userData)
{
	for(unsigned int n = 0; n < context->audioFrames; n++) {
		if(gAudioFramesPerAnalogFrame && !(n % gAudioFramesPerAnalogFrame)) {
			
			uint64_t frames = context->audioFramesElapsed/gAudioFramesPerAnalogFrame + n/gAudioFramesPerAnalogFrame;
			Bela_getDefaultWatcherManager()->tick(frames); // watcher timestamps
			
			pot1 = analogRead(context,  n/gAudioFramesPerAnalogFrame, gPot1Ch);
			pot2 = analogRead(context,  n/gAudioFramesPerAnalogFrame, gPot2Ch);
			
		}
	}
}
```

AS you can see, we are "ticking" the Bela watcher once per every analog frame. When we request variable `pot1` with a period 1000, we will be getting the value of `pot1` every 1000 analog frames. For more advanced timestamping methods, you can check the `4_Sparse_timestamping.ipynb` notebook.

### Monitoring a fixed number of values
Alternatively, you can ask the monitor to monitor a variable for a fixed number of values. Another difference between the Streamer and the Monitor is that the Monitor will monitor exactly `n_values`. This is because in the monitor, the data buffers sent by Bela have only one timestamped value each, whilst in the Streamer, the buffers have a larger number of data points (which depends on the timestamping method and the data type). Let's see an example:

In [8]:
monitored_data = monitor.monitor_n_values(variables= ["pot1", "pot2"],
                         periods= [1000,2000],
                         n_values= 10)

Monitoring 10 values for variables ['pot1', 'pot2'] with periods [1000, 2000]...
Stopped monitoring variables ['pot1', 'pot2']...


We can check that even when the periods for `pot1` and `pot2` where different, we monitored exactly 10 values for each:

In [21]:
for var in ["pot1", "pot2"]:
    print(f"Monitored values for {var}: {len(monitored_data[var])}")

Monitored values for pot1: 10
Monitored values for pot2: 10


In [9]:
df = pd.DataFrame(monitor.values["pot1"])

df.head()

,timestamps,values
0,1269744,0.451141
1,1270744,0.451096
2,1271744,0.450974
3,1272744,0.450974
4,1273744,0.451080


In [10]:
df = pd.DataFrame(monitor.values["pot2"])

df.head()

,timestamps,values
0,1260840,0.493332
1,1262840,0.493393
2,1264840,0.493256
3,1266840,0.493225
4,1268840,0.493271


### Saving monitored data
You can also store the monitored values by passing `saving_enabled=True`  to either `start_monitoring()` or `monitor_n_values()`. 

In [11]:
monitor.start_monitoring(variables= ["pot1", "pot2"],periods= [600,2000], saving_enabled=True)
await asyncio.sleep(1)
monitor.stop_monitoring()

Started monitoring variables ['pot1', 'pot2']... Run stop_monitoring() to stop monitoring.
Stopped monitoring variables ['pot1', 'pot2']...


{'pot1': {'timestamps': [1508840,
   1509440,
   1510040,
   1510640,
   1511240,
   1511840,
   1512440,
   1513040,
   1513640,
   1514240],
  'values': [0.4510498046875,
   0.4511260986328125,
   0.451141357421875,
   0.4510955810546875,
   0.4511566162109375,
   0.4510040283203125,
   0.451141357421875,
   0.4510040283203125,
   0.45123291015625,
   0.451080322265625]},
 'pot2': {'timestamps': [1496640,
   1498640,
   1500640,
   1502640,
   1504640,
   1506640,
   1508640,
   1510640,
   1512640,
   1514640],
  'values': [0.493255615234375,
   0.4932098388671875,
   0.4933319091796875,
   0.4932403564453125,
   0.4932708740234375,
   0.4932098388671875,
   0.49322509765625,
   0.4932708740234375,
   0.4933319091796875,
   0.4932403564453125]}}

You can load the data using `load_data_from_file()`:

In [12]:
pot1_saved_data = monitor.load_data_from_file("pot1_monitor.txt")

df = pd.DataFrame(pot1_saved_data)
df.head()

,timestamps,values
0,1492640,0.450912
1,1493240,0.451080
2,1493840,0.451111
3,1494440,0.451019
4,1495040,0.451050


### Peeking at variables
You can also use the monitor to "peek" at variables, that is, requesting a single value.

In [13]:
peeked_values = monitor.peek() # peeks at all the available variables (otherwise specify)
peeked_values["pot1"]

Peeking at variables ['pot1', 'pot2']...


{'timestamp': 2820200, 'value': 0.4510650634765625}